In [1]:
import torch
import os
import cv2
import numpy as np

In [ ]:
# YOLOv5 모델 로드 (사전 학습된 모델 사용)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # 'yolov5s' 등 원하는 모델 선택 가능

In [3]:
# 입력 및 출력 경로 설정
input_dirs = [
    '/home/gyuha_lee/DCC2024/dataset/training_image',
]
output_dirs = [
    '/home/gyuha_lee/DCC2024/dataset/bg_remove/yolov5/rm_more/training_image_no_bg',
]

In [4]:
# 출력 디렉토리 생성
for output_dir in output_dirs:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

In [ ]:
# 모든 이미지에 대해 YOLO 모델 적용 및 마스킹
for input_dir, output_dir in zip(input_dirs, output_dirs):
    for img_file in os.listdir(input_dir):
        if img_file.endswith('.jpg'):
            input_path = os.path.join(input_dir, img_file)
            output_path = os.path.join(output_dir, img_file)

            # 이미지 불러오기
            image = cv2.imread(input_path)

            # 모델 추론
            results = model(input_path)

            # 탐지 결과에서 마스크 생성
            detected = results.pandas().xyxy[0]
            mask = np.zeros(image.shape[:2], dtype=np.uint8)  # 빈 마스크 생성

            for index, row in detected.iterrows():
                # 옷과 관련된 객체만 필터링 (필요에 따라 추가)
                if row['name'] in ['person', 'shirt', 'pants', 'dress']:
                    x_min, y_min, x_max, y_max = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                    # 탐지된 객체 영역에 대해 마스크 설정
                    mask[y_min:y_max, x_min:x_max] = 255

            # 원본 이미지에 마스크 적용
            masked_image = cv2.bitwise_and(image, image, mask=mask)

            # 알파 채널 추가 (배경을 투명하게 처리)
            b, g, r = cv2.split(masked_image)
            alpha = mask  # 마스크를 알파 채널로 사용
            masked_image_rgba = cv2.merge([b, g, r, alpha])

            # 결과 저장
            output_path = output_path.replace('.jpg', '.png')  # 알파 채널을 포함하므로 .png로 저장
            cv2.imwrite(output_path, masked_image_rgba)


In [ ]:
print("모든 이미지에 대해 배경 마스킹 작업이 완료되었습니다.")